## Trying parallel overscan subtraction on AuxTel ISR

Craig Lage - 04-Jan-23

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
import lsst.afw.cameraGeom.utils as camGeomUtils
from lsst.ip.isr import IsrTask, IsrTaskConfig
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm

from lsst.summit.utils import quickSmooth
%matplotlib inline

from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.meas.algorithms.installGaussianPsf import InstallGaussianPsfTask

In [ ]:
from lsst.daf.butler import Butler
butler = Butler('/repo/embargo', collections=["LATISS/raw/all", "u/cslage/sdf/calib/latiss/calib_par_cross_20230113"])

In [ ]:
def plotExp(exp):
    # Borrowed from summit utils
    data = quickSmooth(exp.image.array, 1)
    vmin = np.nanpercentile(data, 1)
    vmax = np.nanpercentile(data, 99)
    print(vmin, vmax)
    figure = plt.figure(figsize=(12,12))
    figure.clear()
    ax1 = figure.add_subplot(111)
    im1 = ax1.imshow(data, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
    ax1.tick_params(which="major", direction="in", top=True, right=True, labelsize=8)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im1, cax=cax)
    plt.tight_layout()
    #plt.show()
    return figure

 
def repairCosmics(postIsr):
    # Borrowed from summit utils
    if postIsr.getPsf() is None:
        installPsfTask = InstallGaussianPsfTask()
        installPsfTask.run(postIsr)

    # TODO: try adding a reasonably wide Gaussian as a temp PSF
    # and then just running repairTask on its own without any
    # imChar. It should work, and be faster.
    repairConfig = CharacterizeImageTask.ConfigClass()
    repairConfig.doMeasurePsf = False
    repairConfig.doApCorr = False
    repairConfig.doDeblend = False
    repairConfig.doWrite = False
    repairConfig.repair.cosmicray.nCrPixelMax = 200000
    repairTask = CharacterizeImageTask(config=repairConfig)
    repairTask.repair.run(postIsr)
    return

# Run the same image as before, with parallel overscan subtraction


In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.overscan.fitType="MEDIAN_PER_ROW"
isrConfig.overscan.doParallelOverscan=True
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=True
isrConfig.doVariance=True
isrConfig.doLinearize=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=True
isrConfig.doFringe=False
isrConfig.doApplyGains=False
isrConfig.doDefect=True
isrConfig.doNanMasking=True
isrConfig.doInterpolate=True
isrConfig.doSaturation=True
isrConfig.doSaturationInterpolation=True
isrConfig.doWrite=False
isrTask = IsrTask(config=isrConfig)

In [ ]:
expId=2022121200752
exp = butler.get('raw', detector=0, exposure=expId)
biasExp = butler.get('bias', detector=0, exposure=expId)
flatExp = butler.get('flat', detector=0, exposure=expId)
darkExp = butler.get('dark', detector=0, exposure=expId)
defectExp = butler.get('defects', detector=0, exposure=expId)
isrResult = isrTask.run(exp, bias=biasExp, flat=flatExp, dark=darkExp, defects=defectExp)
postIsrExp = isrResult.exposure
repairCosmics(postIsrExp)
fig = plotExp(postIsrExp)
plt.savefig(f"/home/c/cslage/u/AuxTel/isr/isr_study_04jan23/Parallel_Overscan_{expId}_13Jan23.png")

In [ ]:
pixelHist = postIsrExp.image.array.flatten()
pixelHist = np.clip(pixelHist, -40.0,40.0)
mean = np.mean(pixelHist)
std = np.std(pixelHist)
plt.title("Parallel Overscan ISR pixel distribution")
plt.hist(pixelHist, range=(-40,40))
plt.text(-30.0, 5.0E6, f"Mean = {mean:.2f}")
plt.text(-30.0, 4.0E6, f"Std = {std:.2f}")
plt.savefig(f"/home/c/cslage/u/AuxTel/isr/isr_study_04jan23/Parallel_Overscan_Sigma_13Jan23.pdf")

In [ ]:
plt.imshow(postIsrExp.image.array[3500:4000, 550:650], interpolation='nearest',vmin=1, vmax=100)

 ### Look at median signal and master bias

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.doLinearize=False
isrConfig.overscan.fitType="MEDIAN_PER_ROW"
isrConfig.doOverscan=True
isrConfig.overscan.doParallelOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=False
isrConfig.doVariance=True
isrConfig.doLinearize=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=False
isrConfig.doDefect=False
isrConfig.doNanMasking=True
isrConfig.doInterpolate=True
isrConfig.doSaturation=True
isrConfig.doSaturationInterpolation=True
isrTask = IsrTask(config=isrConfig)

In [ ]:
fig = plt.figure()
plt.subplots_adjust(hspace=0.5, wspace=0.5)

exp = butler.get('raw', detector=0, exposure=expId)
isrResult = isrTask.run(exp)
rawPlot = np.median(isrResult.exposure.image.array[2100:3900, 0:4000], axis=0)
rawPlot -= np.median(rawPlot)
biasPlot = np.median(biasExp.image.array[2100:3900, 0:4000], axis=0)
biasPlot -= np.median(biasPlot)
plt.subplot(2,1,1)
plt.title(f"Parallel Overscan Debug - Top")
plt.plot(rawPlot, label="Raw")
plt.plot(biasPlot, ls = '--', label="MasterBias")
for i in range(8):
    x = 509 * i
    ymin = -10
    ymax = 10
    plt.plot([x, x], [ymin, ymax], ls = '--', lw=1, color = 'black')
plt.xlim(0,4000)
plt.ylim(-10,10)
plt.legend()
plt.xlabel("X coord (pixels)")
plt.ylabel("ADU")
rawPlot = np.median(isrResult.exposure.image.array[100:1900, 0:4000], axis=0)
rawPlot -= np.median(rawPlot)
biasPlot = np.median(biasExp.image.array[100:1900, 0:4000], axis=0)
biasPlot -= np.median(biasPlot)
plt.subplot(2,1,2)
plt.title(f"Parallel Overscan Debug - Bottom")
plt.plot(rawPlot, label="Raw")
plt.plot(biasPlot, ls = '--', label="MasterBias")
for i in range(8):
    x = 509 * i
    ymin = -10
    ymax = 10
    plt.plot([x, x], [ymin, ymax], ls = '--', lw=1, color = 'black')
plt.xlim(0,4000)
plt.ylim(-10,10)
plt.legend()
plt.xlabel("X coord (pixels)")
plt.ylabel("ADU")

fig.savefig(f"/home/c/cslage/u/AuxTel/isr/isr_study_04jan23/Parallel_Overscan_Bias_Stability_{expId}_06Jan23.png")

In [ ]:
maskIm = postIsrExp.getMaskedImage()

In [ ]:
maskIm.getMask().array.shape

In [ ]:
def plotMask(exp):
    mask = exp.getMaskedImage().getMask().array
    figure = plt.figure(figsize=(10,10))
    figure.clear()
    ax1 = figure.add_subplot(111)
    im1 = ax1.imshow(mask, cmap='gray', origin='lower', interpolation='nearest')
    ax1.tick_params(which="major", direction="in", top=True, right=True, labelsize=8)


In [ ]:
plotMask(postIsrExp)